In [4]:
import pandas as pd
import json
import numpy as np
from collections import defaultdict
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk

# Télécharger les ressources NLTK nécessaires (à faire une seule fois)
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Téléchargement des ressources NLTK...")
    nltk.download('punkt')
    nltk.download('punkt_tab')
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    print("✅ Ressources NLTK téléchargées!")

# ==================== CONFIGURATION ====================
test_captions_path = "outputs/test_captions.csv"
generated_captions_path = "outputs/generated_captions/generated_captions_ft_vgg_dense_transformer.csv"
nom_model = "ft_vgg_dense_transformer"
output_json = f"evaluations/evaluation_model_{nom_model}.json"

# ==================== CHARGEMENT DES DONNÉES ====================
print("Chargement des données...")

# Charger les captions de référence (AVEC skiprows=1 pour ignorer le header)
test_df = pd.read_csv(test_captions_path, engine='python', sep='|', 
                      names=['image', 'caption'], skiprows=1)
print(f"Captions de référence chargées: {len(test_df)} lignes")

# Charger les captions générées
gen_df = pd.read_csv(generated_captions_path, engine='python', sep='|', 
                     names=['image', 'caption'])
print(f"Captions générées chargées: {len(gen_df)} lignes")

# Organiser les captions de référence par image
references = defaultdict(list)
for _, row in test_df.iterrows():
    img_name = row['image'].strip()
    caption = row['caption'].strip()
    references[img_name].append(caption)

print(f"Nombre d'images uniques: {len(references)}")

# Organiser les captions générées
generated = {}
for _, row in gen_df.iterrows():
    img_name = row['image'].strip()
    caption = row['caption'].strip()
    generated[img_name] = caption

print(f"Nombre de captions générées: {len(generated)}")

# ==================== IMPORT PYCOCOEVALCAP ====================
try:
    from pycocoevalcap.cider.cider import Cider
    from pycocoevalcap.spice.spice import Spice
    CIDER_AVAILABLE = True
    SPICE_AVAILABLE = True
    print("✅ CIDEr et SPICE disponibles")
except ImportError:
    print("⚠️  pycocoevalcap non installé.")
    CIDER_AVAILABLE = False
    SPICE_AVAILABLE = False

# ==================== FONCTIONS D'ÉVALUATION ====================

def tokenize(text):
    """Tokenize le texte en mots"""
    return nltk.word_tokenize(text.lower())

def calculate_bleu_scores(references_list, hypothesis):
    """Calcule BLEU-1 à BLEU-4"""
    smooth = SmoothingFunction()
    
    # Tokenizer
    ref_tokens = [tokenize(ref) for ref in references_list]
    hyp_tokens = tokenize(hypothesis)
    
    bleu1 = sentence_bleu(ref_tokens, hyp_tokens, weights=(1, 0, 0, 0), 
                          smoothing_function=smooth.method1)
    bleu2 = sentence_bleu(ref_tokens, hyp_tokens, weights=(0.5, 0.5, 0, 0), 
                          smoothing_function=smooth.method1)
    bleu3 = sentence_bleu(ref_tokens, hyp_tokens, weights=(0.33, 0.33, 0.33, 0), 
                          smoothing_function=smooth.method1)
    bleu4 = sentence_bleu(ref_tokens, hyp_tokens, weights=(0.25, 0.25, 0.25, 0.25), 
                          smoothing_function=smooth.method1)
    
    return bleu1, bleu2, bleu3, bleu4

def calculate_meteor(references_list, hypothesis):
    """Calcule METEOR score"""
    ref_tokens = [tokenize(ref) for ref in references_list]
    hyp_tokens = tokenize(hypothesis)
    
    # METEOR nécessite une seule référence à la fois, on prend la moyenne
    scores = [meteor_score([ref], hyp_tokens) for ref in ref_tokens]
    return np.mean(scores)

def calculate_rouge_l(references_list, hypothesis):
    """Calcule ROUGE-L score"""
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    
    scores = []
    for ref in references_list:
        score = scorer.score(ref, hypothesis)
        scores.append(score['rougeL'].fmeasure)
    
    return np.mean(scores)

def calculate_f1_score(references_list, hypothesis):
    """Calcule F1-score basé sur les mots communs"""
    hyp_words = set(tokenize(hypothesis))
    
    f1_scores = []
    for ref in references_list:
        ref_words = set(tokenize(ref))
        
        if len(hyp_words) == 0 and len(ref_words) == 0:
            f1_scores.append(1.0)
            continue
        
        common = hyp_words.intersection(ref_words)
        
        if len(common) == 0:
            f1_scores.append(0.0)
            continue
        
        precision = len(common) / len(hyp_words) if len(hyp_words) > 0 else 0
        recall = len(common) / len(ref_words) if len(ref_words) > 0 else 0
        
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

def prepare_for_coco_eval(references_dict, generated_dict, common_images):
    """Prépare les données au format attendu par pycocoevalcap"""
    gts = {}
    res = {}
    
    for idx, img_name in enumerate(common_images):
        # Format: liste de strings directement
        gts[idx] = references_dict[img_name]
        res[idx] = [generated_dict[img_name]]
    
    return gts, res

# ==================== CALCUL DES MÉTRIQUES ====================
print("\nCalcul des métriques...")

metrics = {
    'BLEU-1': [],
    'BLEU-2': [],
    'BLEU-3': [],
    'BLEU-4': [],
    'METEOR': [],
    'ROUGE-L': [],
    'F1-Score': []
}

# Filtrer les images qui ont à la fois des références et des prédictions
common_images = list(set(references.keys()).intersection(set(generated.keys())))
print(f"Nombre d'images à évaluer: {len(common_images)}")

# Calcul des métriques classiques
for img_name in common_images:
    refs = references[img_name]
    hyp = generated[img_name]
    
    # BLEU scores
    bleu1, bleu2, bleu3, bleu4 = calculate_bleu_scores(refs, hyp)
    metrics['BLEU-1'].append(bleu1)
    metrics['BLEU-2'].append(bleu2)
    metrics['BLEU-3'].append(bleu3)
    metrics['BLEU-4'].append(bleu4)
    
    # METEOR
    meteor = calculate_meteor(refs, hyp)
    metrics['METEOR'].append(meteor)
    
    # ROUGE-L
    rouge_l = calculate_rouge_l(refs, hyp)
    metrics['ROUGE-L'].append(rouge_l)
    
    # F1-Score
    f1 = calculate_f1_score(refs, hyp)
    metrics['F1-Score'].append(f1)

# Calcul CIDEr
if CIDER_AVAILABLE:
    print("\nCalcul de CIDEr...")
    try:
        gts, res = prepare_for_coco_eval(references, generated, common_images)
        cider_scorer = Cider()
        cider_score, _ = cider_scorer.compute_score(gts, res)
        metrics['CIDEr'] = [cider_score]
        print(f"✅ CIDEr: {cider_score:.4f}")
    except Exception as e:
        print(f"❌ Erreur CIDEr: {e}")
        CIDER_AVAILABLE = False

# Calcul SPICE
if SPICE_AVAILABLE:
    print("\nCalcul de SPICE (peut prendre du temps)...")
    try:
        gts, res = prepare_for_coco_eval(references, generated, common_images)
        spice_scorer = Spice()
        spice_score, _ = spice_scorer.compute_score(gts, res)
        metrics['SPICE'] = [spice_score]
        print(f"✅ SPICE: {spice_score:.4f}")
    except Exception as e:
        print(f"❌ Erreur SPICE: {e}")
        print("Note: SPICE nécessite Java.")
        SPICE_AVAILABLE = False

# ==================== CALCUL DES MOYENNES ====================
results = {
    'model_name': nom_model,
    'num_images_evaluated': len(common_images),
    'metrics': {}
}

for metric_name, values in metrics.items():
    if len(values) > 0:
        results['metrics'][metric_name] = {
            'mean': float(np.mean(values)),
            'std': float(np.std(values)) if len(values) > 1 else 0.0,
            'min': float(np.min(values)),
            'max': float(np.max(values))
        }

# ==================== SAUVEGARDE DES RÉSULTATS ====================
print(f"\nSauvegarde des résultats dans {output_json}...")

with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(results, indent=4, fp=f)

# ==================== AFFICHAGE DES RÉSULTATS ====================
print("\n" + "="*60)
print(f"RÉSULTATS D'ÉVALUATION - {nom_model}")
print("="*60)
print(f"Nombre d'images évaluées: {len(common_images)}")
print("\nMétriques moyennes:")
print("-"*60)

metric_order = ['BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', 'METEOR', 'ROUGE-L', 'CIDEr', 'SPICE', 'F1-Score']

for metric_name in metric_order:
    if metric_name in results['metrics']:
        mean_val = results['metrics'][metric_name]['mean']
        std_val = results['metrics'][metric_name]['std']
        print(f"{metric_name:12s}: {mean_val:.4f} ± {std_val:.4f}")

print("="*60)
print(f"\n✅ Résultats sauvegardés dans: {output_json}")

Chargement des données...
Captions de référence chargées: 4050 lignes
Captions générées chargées: 810 lignes
Nombre d'images uniques: 810
Nombre de captions générées: 810
✅ CIDEr et SPICE disponibles

Calcul des métriques...
Nombre d'images à évaluer: 810

Calcul de CIDEr...
✅ CIDEr: 0.4929

Calcul de SPICE (peut prendre du temps)...


Parsing reference captions
Parsing test captions


SPICE evaluation took: 2.880 s
✅ SPICE: 0.1391

Sauvegarde des résultats dans evaluations/evaluation_model_ft_vgg_dense_transformer.json...

RÉSULTATS D'ÉVALUATION - ft_vgg_dense_transformer
Nombre d'images évaluées: 810

Métriques moyennes:
------------------------------------------------------------
BLEU-1      : 0.6145 ± 0.1829
BLEU-2      : 0.4039 ± 0.2264
BLEU-3      : 0.2683 ± 0.2173
BLEU-4      : 0.1779 ± 0.1822
METEOR      : 0.2836 ± 0.1292
ROUGE-L     : 0.3196 ± 0.1275
CIDEr       : 0.4929 ± 0.0000
SPICE       : 0.1391 ± 0.0000
F1-Score    : 0.3701 ± 0.1181

✅ Résultats sauvegardés dans: evaluations/evaluation_model_ft_vgg_dense_transformer.json


test

In [5]:
# === CELLULE DE TEST ===
print("="*60)
print("TESTS DE VÉRIFICATION")
print("="*60)

# Test 1 : Vérifier le chargement
print(f"\n✓ Test 1 - Chargement des données")
print(f"  - Références : {len(test_df)} lignes")
print(f"  - Images uniques : {len(references)}")
print(f"  - Captions générées : {len(generated)}")
print(f"  - Images en commun : {len(common_images)}")

# Test 2 : Afficher quelques exemples
print(f"\n✓ Test 2 - Exemples de données")
sample_img = list(common_images)[0]
print(f"\nImage : {sample_img}")
print(f"Références ({len(references[sample_img])}) :")
for i, ref in enumerate(references[sample_img], 1):
    print(f"  {i}. {ref}")
print(f"\nGénérée :")
print(f"  {generated[sample_img]}")

# Test 3 : Tester une métrique sur un exemple
print(f"\n✓ Test 3 - Test BLEU sur l'exemple")
refs = references[sample_img]
hyp = generated[sample_img]
bleu1, bleu2, bleu3, bleu4 = calculate_bleu_scores(refs, hyp)
print(f"  BLEU-1: {bleu1:.4f}")
print(f"  BLEU-2: {bleu2:.4f}")
print(f"  BLEU-3: {bleu3:.4f}")
print(f"  BLEU-4: {bleu4:.4f}")

# Test 4 : Vérifier que le JSON est bien créé
print(f"\n✓ Test 4 - Vérification du fichier JSON")
import os
if os.path.exists(output_json):
    print(f"  ✅ Fichier créé : {output_json}")
    with open(output_json, 'r') as f:
        saved_results = json.load(f)
    print(f"  ✅ Nombre de métriques sauvegardées : {len(saved_results['metrics'])}")
else:
    print(f"  ❌ Fichier non trouvé")

print("\n" + "="*60)
print("TOUS LES TESTS TERMINÉS ✅")
print("="*60)

TESTS DE VÉRIFICATION

✓ Test 1 - Chargement des données
  - Références : 4050 lignes
  - Images uniques : 810
  - Captions générées : 810
  - Images en commun : 810

✓ Test 2 - Exemples de données

Image : 3041487045_b48ac7ed08.jpg
Références (5) :
  1. A dog runs through the woods .
  2. A pale tan dog romps through a wooded area
  3. A tan dog is carrying a small object in its mouth .
  4. a tan dog running
  5. A yellow dog is trotting through the leaves .

Générée :
  a brown dog is running on a dirt path .

✓ Test 3 - Test BLEU sur l'exemple
  BLEU-1: 0.6000
  BLEU-2: 0.2582
  BLEU-3: 0.0964
  BLEU-4: 0.0587

✓ Test 4 - Vérification du fichier JSON
  ✅ Fichier créé : evaluations/evaluation_model_ft_vgg_dense_transformer.json
  ✅ Nombre de métriques sauvegardées : 9

TOUS LES TESTS TERMINÉS ✅


schema du fichier des captions: 


1022454428_b6b660a67b.jpg|a man in a black shirt and a woman in a white shirt are sitting on a bench .

102351840_323e3de834.jpg|a man is standing on a red board with a man in the background .

    ...